<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Homework/5_Class_Homework/PH_5_Homework_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries

In [71]:
from google.colab import drive
drive.mount('/content/drive')
TWSM_path = "/content/drive/MyDrive/Colab_Notebooks/02_HWR"

#from TWSM import *
from tensorflow import keras
from tensorflow.keras import layers
import spacy
## Import packages
import pickle
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

# run this from a normal command line
#!python -m spacy download en_core_web_md #160MB

#can I download this to a local file instead and load it fom drive?


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install whatlies

https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab

Now, *** restart the colab runtime *** !!

## word2vec in spacy

In [ ]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

# process a sentence using the model
doc = nlp("The sun is shining brightly today but the moon is not")

# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'text':
doc[3].vector

# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector

In [ ]:
#Opposites are not necessarily different
doc = nlp("I loved Narnia but hated Armageddon")

print(doc[1])
print(doc[4])
print(doc[1].similarity(doc[4]))
print(doc[4].similarity(doc[1]))

doc = nlp("The king and the queen are enjoying a sumptious breakfast today")

print(doc[1])
print(doc[4])
print(doc[1].similarity(doc[4]))
print(doc[4].similarity(doc[1]))

loved
hated
0.66889775
0.66889775
king
queen
0.72526103
0.72526103


WhatLies module

In [ ]:
king = nlp.vocab["king"].vector
man = nlp.vocab["man"].vector
queen = nlp.vocab["queen"].vector
woman = nlp.vocab["woman"].vector

#or:
def w2v(w="king"):
  return nlp.vocab[w].vector

king = w2v("king")

In [ ]:
from whatlies import EmbeddingSet
from whatlies.language import SpacyLanguage

lang = SpacyLanguage('en_core_web_md')
words = ['cat', 'dog', 'fish', 'kitten', 'man', 'woman', 'king', 'queen', 'doctor', 'nurse']

emb = lang[words]
emb.plot_interactive(x_axis='man', y_axis='woman')

alt.LayerChart(...)

**Vector Algebra**

seems much more difficult than in gensim

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

queries = [w for w in nlp.vocab if w.is_lower and w.prob >= -15]

def cos_sim(a, b):
    return cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))

def most_similar_vec(vec, count=10):
    by_similarity = sorted(queries, key=lambda w: cos_sim(w.vector, vec), reverse=True)
    return [w.orth_ for w in by_similarity[:count]]

vec = nlp('woman').vector + nlp('king').vector - nlp("man").vector
most_similar_vec(vec)

['king',
 'queen',
 'prince',
 'princes',
 'kings',
 'princess',
 'princesses',
 'mermaid',
 'royal',
 'royals']

In [ ]:
queries

## word2vec in gensim

In [ ]:
import gensim.downloader as api
#https://github.com/RaRe-Technologies/gensim-data
wv = api.load('glove-wiki-gigaword-100')#128MB
#wv = api.load('glove-wiki-gigaword-50')#65MB


[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.67
'car'	'bicycle'	0.69
'car'	'airplane'	0.65
'car'	'cereal'	0.12
'car'	'communism'	0.04


# Tasks

1. Train a word2vec model of dimension $100$ on the IMD data. (considering words appearing in more than 50 documents) Save the model if you like.

2. Compute the embedding for each review (average word2vec)

3. Fit a keras classifier to the embedded reviews. (2 hidden layers of size 40 each) Report/Monitor the accuracy on the test data.

4. Load the bing sentiment dictionary. Compute two separate embeddings for the negative and positive sentiments.

5. Compute the similarity between these two vectors and a few selected reviews. Does it agree with their label?


## Data

**IMD Movie Reviews**

In [ ]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

17473536/17464789 [==============================] - 0s 0us/step


**Decoding reviews back to text**

In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])
decoded_review = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

decoded_review

1654784/1641221 [==============================] - 0s 0us/step


"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [ ]:
# Decode train data
N=len(train_data)
decoded_reviews = ["" for x in range(N)]

for j in range(N):
  decoded_reviews[j] = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in train_data[j]])
  
# Get rid of beginning "?"
decoded_reviews = [review[2:] for review in decoded_reviews]

## Word2Vec
Train a word2vec model of dimension $100$ on the IMD data. (considering words appearing in more than 50 documents) Save the model if you like.

In [ ]:
# Prepare the dataset for the word2vec model
corpus=[doc.split() for doc in decoded_reviews]

# Train the model for embeddings of size 100 considering words appearing in more than 50 documents, default window=5
model = Word2Vec(corpus, size=100, min_count=50)
model.save(TWSM_path+'/00_data/word2vec_imdb.model')


## Average word2vec
Compute the embedding for each review (average word2vec)

In the following we will derive the corpus. Note that word2vec (as opposed to doc2vec) generates one embedding for each word in the document. These then need to be aggregated at a document level. The simplest way is to determine the average over all words, but you can also use other aggregators.

In [ ]:
# Document representation for the text
corpus_w2v=[[model.wv[word] for word in doc if word in model.wv.vocab.keys()] for doc in corpus]
positive=[i for i in range(len(corpus)) if len(corpus_w2v[i])>0]

corpus_w2v2=[corpus_w2v[i] for i in positive]
#data_lemma2=[data_lemma[i] for i in positive]

# Document average representation
corpus_w2v_avg_clean=[sum(words)/len(words) for words in corpus_w2v2]

# This corpus can be used later in clustering and classification tasks
print(corpus_w2v_avg_clean[10])

[-0.07425238  0.06747388  0.39230126 -0.36663225  0.21546191 -0.36204475
  0.07142097  0.07992306  0.03126279 -0.51161313  0.12652317  0.4391952
 -0.16302423 -0.12542012  0.27796707  0.11315694 -0.1846122   0.07238373
  0.45462906  0.01931621  0.1215534   0.03018505 -0.21474141 -0.00561327
  0.24638928 -0.15758626 -0.16836031  0.21521203  0.06390181 -0.07232941
  0.11160904  0.14934357 -0.30050156  0.19315217  0.30240217  0.12886333
  0.15003869  0.05364362  0.10478669  0.14057606  0.18024667 -0.33982483
  0.00996698  0.03019215 -0.03181847 -0.38736826 -0.02504646  0.09713174
  0.2815634   0.46132156 -0.18515831 -0.38660303 -0.09607214 -0.20545292
  0.44870573 -0.02355427  0.39900258 -0.41577318 -0.2690298  -0.14373663
  0.03981441 -0.06252696 -0.4330292  -0.21920864 -0.24917193  0.06513843
  0.6396006   0.5933016   0.10797528 -0.08458997 -0.09392422 -0.44205388
 -0.33044085  0.00815693  0.00944452 -0.29033545 -0.21981075 -0.16393258
 -0.42866853  0.1167855   0.02380345 -0.19003735  0.

## Keras Classifier
Fit a keras classifier to the embedded reviews. (2 hidden layers of size 40 each) Report/Monitor the accuracy on the test data.

In [ ]:
#Define the architecture
clf = keras.Sequential()
clf.add(layers.Dense(40, input_dim=100, activation='relu')) 
clf.add(layers.Dense(40, activation='relu'))
clf.add(layers.Dense(1, activation='sigmoid')) 

In [ ]:
#Complile
clf.compile(optimizer='adam',
    loss='binary_crossentropy', 
    metrics = ['accuracy'])

In [ ]:
#Early stopping
es = keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [ ]:
#Fit
clf.fit(np.array(corpus_w2v_avg_clean), train_labels, epochs=100, validation_split=0.3, callbacks=[es])

Epoch 1/100
547/547 [==============================] - 2s 3ms/step - loss: 0.4607 - accuracy: 0.7850 - val_loss: 0.3850 - val_accuracy: 0.8307
Epoch 2/100
547/547 [==============================] - 1s 2ms/step - loss: 0.3891 - accuracy: 0.8274 - val_loss: 0.3707 - val_accuracy: 0.8392
Epoch 3/100
547/547 [==============================] - 2s 4ms/step - loss: 0.3794 - accuracy: 0.8327 - val_loss: 0.3707 - val_accuracy: 0.8344
Epoch 4/100
547/547 [==============================] - 2s 4ms/step - loss: 0.3767 - accuracy: 0.8332 - val_loss: 0.3887 - val_accuracy: 0.8236
Epoch 5/100
547/547 [==============================] - 2s 4ms/step - loss: 0.3744 - accuracy: 0.8348 - val_loss: 0.3827 - val_accuracy: 0.8312
Epoch 6/100
547/547 [==============================] - 1s 3ms/step - loss: 0.3679 - accuracy: 0.8370 - val_loss: 0.3897 - val_accuracy: 0.8279
Epoch 7/100
547/547 [==============================] - 2s 3ms/step - loss: 0.3641 - accuracy: 0.8387 - val_loss: 0.3644 - val_accuracy: 0.8419

In [ ]:
# Decode test data
N=len(test_data)
decoded_reviews_test = ["" for x in range(N)]

for j in range(N):
  decoded_reviews_test[j] = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in test_data[j]])
  
# Get rid of beginning "?"
decoded_reviews_test = [review[2:] for review in decoded_reviews_test]

# Prepare the dataset for the word2vec model
corpus_test=[doc.split() for doc in decoded_reviews_test]

# Document representation for the text
corpus_w2v_test=[[model.wv[word] for word in doc if word in model.wv.vocab.keys()] for doc in corpus_test]
positive_test=[i for i in range(len(corpus_test)) if len(corpus_w2v_test[i])>0]

corpus_w2v2_test=[corpus_w2v_test[i] for i in positive_test]

# Document average representation
corpus_w2v_avg_clean_test=[sum(words)/len(words) for words in corpus_w2v2_test]

In [ ]:
#check shape
np.array(corpus_w2v_avg_clean_test).shape

(25000, 100)

In [ ]:
results = clf.evaluate(np.array(corpus_w2v_avg_clean_test), test_labels, verbose=0)
print("Accuracy on test_data:", np.round(results[1],2))

Accuracy on test_data: 0.84


## Bing sentiment embeddings
Load the bing sentiment dictionary. Compute two separate embeddings for the negative and positive sentiments.

In [77]:
#Load bing
bing_df = pd.read_csv(TWSM_path+"/00_data/SentimentDictionaries/bing.csv", index_col=0)
bing_df.sample(5)

,word,sentiment
6748,wowing,positive
2612,gallant,positive
130,affably,positive
1524,dire,negative
492,benevolent,positive


In [79]:
# Filter bing df 
pos_df = bing_df[bing_df["sentiment"]=="positive"]
neg_df = bing_df[bing_df["sentiment"]=="negative"]

In [81]:
# Check if no word got lost during filtering ;)
neg_df.shape[0]+pos_df.shape[0] == bing_df.shape[0]

True

In [ ]:
# Create corpus (reshape, because for Word2Vec it has to be in a list of lists format)
corpus_pos = np.array(pos_df["word"]).reshape(len(pos_df),1)
corpus_neg = np.array(neg_df["word"]).reshape(len(neg_df),1)

In [180]:
# Train the model for embeddings of size 100 with min_count=1 (because there is only 1 "sentence")
model_pos = Word2Vec(corpus_pos, size=100, min_count=1)
model_neg = Word2Vec(corpus_neg, size=100, min_count=1)

## Similarity between positive and negative embeddings
Compute the similarity between these two vectors and a few selected reviews. Does it agree with their label?


> ❗ **Please note:** The solution below is probably not correct, but I did not see another way on how to compute the overall similarity between a review and the embedded sentiment

In [263]:
# Average word2vec corpus for positive sentiment 
corpus_w2v_pos=[[model_pos.wv[word] for word in doc if word in model_pos.wv.vocab.keys()] for doc in corpus]
positive=[i for i in range(len(corpus)) if len(corpus_w2v_pos[i])>0]

corpus_w2v2_pos=[corpus_w2v_pos[i] for i in positive]
decoded_reviews_pos=[decoded_reviews[i] for i in positive]

# Document average representation
corpus_w2v_avg_clean_pos=[sum(words)/len(words) for words in corpus_w2v2_pos]

In [258]:
# Average word2vec corpus for negative sentiment 
corpus_w2v_neg=[[model_neg.wv[word] for word in doc if word in model_neg.wv.vocab.keys()] for doc in corpus]
positive=[i for i in range(len(corpus)) if len(corpus_w2v_neg[i])>0]

corpus_w2v2_neg=[corpus_w2v_neg[i] for i in positive]
decoded_reviews_neg=[decoded_reviews[i] for i in positive]

# Document average representation
corpus_w2v_avg_clean_neg=[sum(words)/len(words) for words in corpus_w2v2_neg]

### Similarity to 1st review (positive)

In [268]:
# Check if review is the same (some reviews were lost during averaging since no positive or negative words were included)
decoded_reviews_pos[0] == decoded_reviews_neg[0]

True

In [269]:
print(decoded_reviews_pos[0])
print("True label:", train_labels[0])

this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think

In [232]:
# Positive connotated words in first review
[word for word in corpus[0] if word in model_pos.wv.vocab.keys()]

['brilliant',
 'amazing',
 'loved',
 'witty',
 'great',
 'brilliant',
 'recommend',
 'amazing',
 'good',
 'brilliant',
 'amazing',
 'lovely']

In [231]:
# Negative connotated words in first review
[word for word in corpus[0] if word in model_neg.wv.vocab.keys()]

['sad', 'cry']

In [279]:
print("Max cosine similarity to positive embedding:", np.round(model_pos.wv.cosine_similarities(corpus_w2v_avg_clean_pos[0], model_pos.wv.vectors).max(),2))
print("Avg. cosine similarity to positive embedding:", np.round(model_pos.wv.cosine_similarities(corpus_w2v_avg_clean_pos[0], model_pos.wv.vectors).mean(),2))

Max cosine similarity to positive embedding: 0.66
Avg. cosine similarity to positive embedding: 0.0


In [280]:
print("Max cosine similarity to negative embedding:", np.round(model_neg.wv.cosine_similarities(corpus_w2v_avg_clean_neg[0], model_neg.wv.vectors).max(),2))
print("Avg. cosine similarity to negative embedding:", np.round(model_neg.wv.cosine_similarities(corpus_w2v_avg_clean_neg[0], model_neg.wv.vectors).mean(),2))

Max cosine similarity to negative embedding: 0.67
Avg. cosine similarity to negative embedding: -0.0


### Similarity to 2nd review (negative)

In [285]:
# index of review
i = 1

In [286]:
# Check if review is the same (some reviews were lost during averaging since no positive or negative words were included)
decoded_reviews_pos[i] == decoded_reviews_neg[i]

True

In [287]:
print(decoded_reviews[i])
print("True label:", train_labels[i])

big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ? the hair is big lots of boobs ? men wear those cut ? shirts that show off their ? sickening that men actually wore them and the music is just ? trash that plays over and over again in almost every scene there is trashy music boobs and ? taking away bodies and the gym still doesn't close for ? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then
True label: 0


In [288]:
# Positive connotated words in first review
[word for word in corpus[i] if word in model_pos.wv.vocab.keys()]

['best', 'love', 'best', 'worked', 'charm', 'good']

In [289]:
# Negative connotated words in first review
[word for word in corpus[i] if word in model_neg.wv.vocab.keys()]

['bad',
 'terrible',
 'cheesy',
 'worst',
 'plot',
 'ridiculous',
 'abomination',
 'laughable',
 'showdown',
 'killer',
 'damn',
 'terribly',
 'sickening',
 'funny',
 'sickening',
 'trash',
 'trashy',
 'bad',
 'disaster',
 'bad']

In [290]:
print("Max cosine similarity to positive embedding:", np.round(model_pos.wv.cosine_similarities(corpus_w2v_avg_clean_pos[i], model_pos.wv.vectors).max(),2))
print("Avg. cosine similarity to positive embedding:", np.round(model_pos.wv.cosine_similarities(corpus_w2v_avg_clean_pos[i], model_pos.wv.vectors).mean(),2))

Max cosine similarity to positive embedding: 0.68
Avg. cosine similarity to positive embedding: 0.0


In [291]:
print("Max cosine similarity to negative embedding:", np.round(model_neg.wv.cosine_similarities(corpus_w2v_avg_clean_neg[i], model_neg.wv.vectors).max(),2))
print("Avg. cosine similarity to negative embedding:", np.round(model_neg.wv.cosine_similarities(corpus_w2v_avg_clean_neg[i], model_neg.wv.vectors).mean(),2))

Max cosine similarity to negative embedding: 0.54
Avg. cosine similarity to negative embedding: 0.0


### Similarity to 3rd review (negative)

In [292]:
# index of review
i = 2

In [293]:
# Check if review is the same (some reviews were lost during averaging since no positive or negative words were included)
decoded_reviews_pos[i] == decoded_reviews_neg[i]

True

In [294]:
print(decoded_reviews[i])
print("True label:", train_labels[i])

this has to be one of the worst films of the 1990s when my friends i were watching this film being the target audience it was aimed at we just sat watched the first half an hour with our jaws touching the floor at how bad it really was the rest of the time everyone else in the theatre just started talking to each other leaving or generally crying into their popcorn that they actually paid money they had ? working to watch this feeble excuse for a film it must have looked like a great idea on paper but on film it looks like no one in the film has a clue what is going on crap acting crap costumes i can't get across how ? this is to watch save yourself an hour a bit of your life
True label: 0


In [295]:
# Positive connotated words in first review
[word for word in corpus[i] if word in model_pos.wv.vocab.keys()]

['like', 'great', 'like']

In [296]:
# Negative connotated words in first review
[word for word in corpus[i] if word in model_neg.wv.vocab.keys()]

['worst', 'bad', 'feeble', 'excuse', 'crap', 'crap']

In [305]:
print("Most similar word to avg. embedding:", model_pos.wv.similar_by_vector(corpus_w2v_avg_clean_pos[i], topn=1))
print("Max cosine similarity to positive embedding:", np.round(model_pos.wv.cosine_similarities(corpus_w2v_avg_clean_pos[i], model_pos.wv.vectors).max(),2))
print("Avg. cosine similarity to positive embedding:", np.round(model_pos.wv.cosine_similarities(corpus_w2v_avg_clean_pos[i], model_pos.wv.vectors).mean(),2))

Most similar word to avg. embedding: [('like', 0.913170337677002)]
Max cosine similarity to positive embedding: 0.91
Avg. cosine similarity to positive embedding: 0.0


In [304]:
print("Most similar word to avg. embedding:", model_neg.wv.similar_by_vector(corpus_w2v_avg_clean_neg[i], topn=1))
print("Max cosine similarity to negative embedding:", np.round(model_neg.wv.cosine_similarities(corpus_w2v_avg_clean_neg[i], model_neg.wv.vectors).max(),2))
print("Avg. cosine similarity to negative embedding:", np.round(model_neg.wv.cosine_similarities(corpus_w2v_avg_clean_neg[i], model_neg.wv.vectors).mean(),2))

Most similar word to avg. embedding: [('crap', 0.7057374715805054)]
Max cosine similarity to negative embedding: 0.71
Avg. cosine similarity to negative embedding: 0.0
